In [73]:
import pandas as pd
import numpy as np
import csv
import datetime
import time

### Question 1
Provide an upper and lower bound on ρij. Also, provide a justification for using log-normalized return (ri(t)) instead of regular return (qi(t)).

In [11]:
name_sector = pd.read_csv('finance_data/Name_sector.csv')

In [55]:
name_sector.groupby(['Sector'])['Sector']
grouped = name_sector['Sector'].groupby(name_sector['Sector'])

In [57]:
grouped.size()

Sector
Consumer Discretionary        85
Consumer Staples              37
Energy                        34
Financials                    66
Health Care                   60
Industrials                   65
Information Technology        70
Materials                     25
Real Estate                   31
Telecommunication Services     4
Utilities                     28
Name: Sector, dtype: int64

In [154]:
ticket_symbol = list(name_sector['Symbol'])

In [52]:
name_sector

,Symbol,Sector
0,A,Health Care
1,AAL,Industrials
2,AAP,Consumer Discretionary
3,AAPL,Information Technology
4,ABBV,Health Care
5,ABC,Health Care
6,ABT,Health Care
7,ACN,Information Technology
8,ADBE,Information Technology
9,ADI,Information Technology


In [13]:
stock_data = {}
for stock in ticket_symbol:
    file_name = 'finance_data/data/' + stock + '.csv'
    stock_data[stock] = pd.read_csv(file_name)

In [14]:
stock_data['CFG'].head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2014-09-24,21.500000,23.219999,21.350000,23.080000,60450900,22.001585
1,2014-09-25,22.559999,23.410000,22.450001,23.049999,8430000,21.972986
2,2014-09-26,23.180000,23.370001,22.910000,23.250000,2504400,22.163642
3,2014-09-29,23.139999,23.450001,22.910000,23.230000,4832600,22.144576
4,2014-09-30,23.110001,23.570000,23.049999,23.420000,4103100,22.325698


In [15]:
len(stock_data['A']['Date'])

765

In [165]:
# For each stock, Calculate Qi
stock_Q = {}
for stock_name in stock_data.keys():
    n = len(stock_data[stock_name]['Date'])
    P_ = stock_data[stock_name]['Close']
    Q_ = np.zeros(n-1)
    for t in range(n-1):
        Q_[t] = (P_[t+1] - P_[t])/P_[t]
    stock_Q[stock_name] = Q_

In [17]:
stock_Q['A'][0:5]

array([ 0.00128511,  0.0144848 , -0.00578344,  0.00036355,  0.00490643])

In [18]:
np.log(1+stock_Q['A'][0:5])

array([ 0.00128429,  0.0143809 , -0.00580023,  0.00036349,  0.00489444])

In [19]:
# For each stock, Calculate Ri
stock_R = {}
for stock_name in stock_data.keys():
    n = len(stock_data[stock_name]['Date'])
    P_ = stock_data[stock_name]['Close']
    R_ = np.zeros(n-1)
    for t in range(n-1):
        R_[t] = np.log(1 + (P_[t+1] - P_[t])/P_[t])
    stock_R[stock_name] = R_

In [20]:
stock_R['A'][0:5]

array([ 0.00128429,  0.0143809 , -0.00580023,  0.00036349,  0.00489444])

In [21]:
# 505 stock in total
len(stock_data.keys())

505

In [22]:
# Un-normalized return qi(t)
ro0 = np.zeros([505, 505])
i = 0
for stock_i in stock_data.keys():
    j = 0
    for stock_j in stock_data.keys():
        qi = list(stock_Q[stock_i])
        qj = list(stock_Q[stock_j])
        n = abs(len(qi) - len(qj))
        if len(qi) < len(qj):
            qi = [0] * n + qi
        elif len(qi) > len(qj):
            qj = [0] * n + qj
        upper = np.mean(np.multiply(qi, qj)) - np.mean(qi)*np.mean(qj)
        temp1 = np.mean(np.multiply(qi, qi)) - np.mean(qi)*np.mean(qi)
        temp2 = np.mean(np.multiply(qj, qj)) - np.mean(qj)*np.mean(qj)
        lower = np.sqrt(temp1*temp2)
        ro0[i, j] = upper / lower
        j += 1
    i += 1

In [23]:
upper_bound = np.max(ro0)
lower_bound = np.min(ro0)
print('upper bound: ', upper_bound)
print('lower bound: ', lower_bound)

upper bound:  1.0
lower bound:  -0.17778692106048652


In [10]:
# Log-normalized return ri(t)
ro = np.zeros([505, 505])
i = 0
for stock_i in stock_data.keys():
    j = 0
    for stock_j in stock_data.keys():
        ri = list(stock_R[stock_i])
        rj = list(stock_R[stock_j])
        n = abs(len(ri) - len(rj))
        if len(ri) < len(rj):
            ri = [0] * n + ri
        elif len(ri) > len(rj):
            rj = [0] * n + rj
        upper = np.mean(np.multiply(ri, rj)) - np.mean(ri)*np.mean(rj)
        temp1 = np.mean(np.multiply(ri, ri)) - np.mean(ri)*np.mean(ri)
        temp2 = np.mean(np.multiply(rj, rj)) - np.mean(rj)*np.mean(rj)
        lower = np.sqrt(temp1*temp2)
        ro[i, j] = ro[j, i] = upper / lower
        j += 1
    i += 1

In [17]:
ro = np.matrix(ro)

In [36]:
upper_bound_log = np.max(ro)
lower_bound_log = np.min(ro)
print('upper bound log normalized: ', upper_bound_log)
print('lower bound log normalized: ', lower_bound_log)

upper bound log normalized:  1.0
lower bound log normalized:  -0.1985733513184204


### Question 2
Plot the degree distribution of the correlation graph and a histogram showing the un-normalized distribution of edge weights.

In [24]:
# Calculate weights
W = np.zeros([505, 505])
for i in range(505):
    for j in range(505):
        W[i, j] = np.sqrt(2 * (1 - ro0[i, j]))

In [30]:
W.shape

(505, 505)

In [31]:
W[0,0:5]

array([0.        , 1.17679472, 1.26092422, 1.31279805, 1.11236814])

In [25]:
np.savetxt('weight.csv', W, delimiter = ',')  

In [43]:
with open('eggs.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(['fromID', 'toID', 'weight'])
    i = 0
    for stock_i in stock_data.keys():
        j = 0
        for stock_j in stock_data.keys():
            spamwriter.writerow([stock_i, stock_j, W[i, j]])
            j += 1
        i += 1

In [44]:
weight = pd.read_csv('weight.csv', header = None)

In [45]:
weight.shape

(505, 505)

In [46]:
weight.columns = ticket_symbol

In [41]:
weight.index = ticket_symbol

In [47]:
weight.head(10)

,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,XLNX,XOM,XRAY,XRX,XYL,YHOO,YUM,ZBH,ZION,ZTS
0,0.000000,1.176795,1.260924,1.312798,1.112368,1.235005,0.997387,1.034910,1.081475,1.090180,...,1.119880,1.105590,1.056239,1.149483,1.025371,1.127184,1.195845,1.076359,1.061776,1.129201
1,1.176795,0.000000,1.217204,1.319783,1.221631,1.245923,1.134056,1.170589,1.155097,1.159290,...,1.210412,1.281125,1.171184,1.183066,1.170115,1.177826,1.215768,1.182905,1.136932,1.202120
2,1.260924,1.217204,0.000000,1.338574,1.235446,1.295961,1.167519,1.218371,1.198511,1.223334,...,1.220717,1.245289,1.196178,1.230834,1.169921,1.236769,1.218461,1.194415,1.177211,1.248967
3,1.312798,1.319783,1.338574,0.000000,1.302351,1.344770,1.296632,1.303328,1.292056,1.347187,...,1.323811,1.308664,1.306114,1.327523,1.322185,1.311656,1.313634,1.311200,1.321305,1.326493
4,1.112368,1.221631,1.235446,1.302351,0.000000,1.116808,1.035537,1.213882,1.172646,1.187358,...,1.189425,1.187680,1.136476,1.219951,1.169494,1.207856,1.268482,1.071282,1.191775,1.084432
5,1.235005,1.245923,1.295961,1.344770,1.116808,0.000000,1.106692,1.225549,1.254013,1.272620,...,1.270320,1.270577,1.191295,1.301992,1.251111,1.269412,1.283453,1.247031,1.228677,1.148446
6,0.997387,1.134056,1.167519,1.296632,1.035537,1.106692,0.000000,1.005614,1.052807,1.093915,...,1.096148,1.114816,0.984107,1.120821,1.049133,1.126044,1.188509,0.956318,1.070749,1.040966
7,1.034910,1.170589,1.218371,1.303328,1.213882,1.225549,1.005614,0.000000,1.008162,1.054595,...,1.108071,1.083170,1.021742,1.076726,0.989748,1.109468,1.163295,1.124515,1.062786,1.100973
8,1.081475,1.155097,1.198511,1.292056,1.172646,1.254013,1.052807,1.008162,0.000000,1.046167,...,1.077059,1.190927,1.091797,1.165679,1.080357,1.113403,1.102204,1.111356,1.132788,1.157894
9,1.090180,1.159290,1.223334,1.347187,1.187358,1.272620,1.093915,1.054595,1.046167,0.000000,...,0.855037,1.116383,1.121749,1.127626,1.048248,1.169698,1.185849,1.159400,1.068277,1.156922


In [48]:
weight.to_csv('weight1.csv')

### Question 5

In [160]:
# Sample data
stock_data_sampled = {}
for stock in ticket_symbol:
    print(stock)
    file_name = 'finance_data/data/' + stock + '.csv'
    temp_df = pd.read_csv(file_name)
    drop_list = []
    for i in range(temp_df.shape[0]):
        #print(i)
        string = temp_df['Date'][i]
        #print(string)
        if '/' in string:
            date_time = datetime.datetime.strptime(string,'%m/%d/%Y').weekday()
        else:
            date_time = datetime.datetime.strptime(string,'%Y-%m-%d').weekday()
        if date_time != 0:
            drop_list.append(i)
    #print(len(drop_list))
    df = temp_df.drop(drop_list)
    stock_data_sampled[stock] = df

A
AAL
AAP
AAPL
ABBV
ABC
ABT
ACN
ADBE
ADI
ADM
ADP
ADS
ADSK
AEE
AEP
AES
AET
AFL
AGN
AIG
AIV
AIZ
AJG
AKAM
ALB
ALK
ALL
ALLE
ALXN
AMAT
AMD
AME
AMG
AMGN
AMP
AMT
AMZN
AN
ANTM
AON
APA
APC
APD
APH
ARE
ARNC
ATVI
AVB
AVGO
AVY
AWK
AXP
AYI
AZO
BA
BAC
BAX
BBBY
BBT
BBY
BCR
BDX
BEN
BF.B
BHI
BIIB
BK
BLK
BLL
BMY
BRK.B
BSX
BWA
BXP
C
CA
CAG
CAH
CAT
CB
CBG
CBOE
CBS
CCI
CCL
CELG
CERN
CF
CFG
CHD
CHK
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COG
COH
COL
COO
COP
COST
COTY
CPB
CRM
CSCO
CSRA
CSX
CTAS
CTL
CTSH
CTXS
CVS
CVX
CXO
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLPH
DLR
DLTR
DOV
DOW
DPS
DRI
DTE
DUK
DVA
DVN
DXC
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
EOG
EQIX
EQR
EQT
ES
ESRX
ESS
ETFC
ETN
ETR
EVHC
EW
EXC
EXPD
EXPE
EXR
F
FAST
FB
FBHS
FCX
FDX
FE
FFIV
FIS
FISV
FITB
FL
FLIR
FLR
FLS
FMC
FOX
FOXA
FRT
FTI
FTV
GD
GE
GGP
GILD
GIS
GLW
GM
GOOG
GOOGL
GPC
GPN
GPS
GRMN
GS
GT
GWW
HAL
HAS
HBAN
HBI
HCA
HCN
HCP
HD
HES
HIG
HOG
HOLX
HON
HP
HPE
HPQ
HRB
HRL
HRS
HSIC
HST
HSY
HUM
IBM
ICE
IDXX
IFF
ILMN
INCY


In [162]:
len(stock_data_sampled['A']['Date'])

143

In [167]:
len(stock_data_sampled.keys())

505

In [168]:
n = len(stock_data_sampled['A']['Date'])

In [169]:
n

143

In [176]:
P_ = np.array(stock_data_sampled['A']['Close'])

In [171]:
Q_ = np.zeros(n-1)

In [178]:
for t in range(142):
        Q_[t] = (P_[t+1] - P_[t])/P_[t]

In [179]:
# For each stock, Calculate weekly Qi_
stock_Q_week = {}
for stock_name in stock_data_sampled.keys():
    n = len(stock_data_sampled[stock_name]['Date'])
    P_ = np.array(stock_data_sampled[stock_name]['Close'])
    Q_ = np.zeros(n-1)
    for t in range(n-1):
        Q_[t] = (P_[t+1] - P_[t])/P_[t]
    stock_Q_week[stock_name] = Q_

In [180]:
# For each stock, Calculate weekly Ri_
stock_R_week = {}
for stock_name in stock_data_sampled.keys():
    n = len(stock_data_sampled[stock_name]['Date'])
    P_ = np.array(stock_data_sampled[stock_name]['Close'])
    R_ = np.zeros(n-1)
    for t in range(n-1):
        R_[t] = np.log(1 + (P_[t+1] - P_[t])/P_[t])
    stock_R_week[stock_name] = R_

In [182]:
len(stock_Q_week['A'])

142

In [183]:
# Un-normalized return qi(t)
ro0_week = np.zeros([505, 505])
i = 0
for stock_i in stock_data.keys():
    j = 0
    for stock_j in stock_data.keys():
        qi = list(stock_Q_week[stock_i])
        qj = list(stock_Q_week[stock_j])
        n = abs(len(qi) - len(qj))
        if len(qi) < len(qj):
            qi = [0] * n + qi
        elif len(qi) > len(qj):
            qj = [0] * n + qj
        upper = np.mean(np.multiply(qi, qj)) - np.mean(qi)*np.mean(qj)
        temp1 = np.mean(np.multiply(qi, qi)) - np.mean(qi)*np.mean(qi)
        temp2 = np.mean(np.multiply(qj, qj)) - np.mean(qj)*np.mean(qj)
        lower = np.sqrt(temp1*temp2)
        ro0_week[i, j] = upper / lower
        j += 1
    i += 1

In [186]:
# Calculate weekly weights
W_week = np.zeros([505, 505])
for i in range(505):
    for j in range(505):
        W_week[i, j] = np.sqrt(2 * (1 - ro0_week[i, j]))

In [187]:
np.savetxt('weight_weekly.csv', W_week, delimiter = ',')  

In [188]:
weight = pd.read_csv('weight_weekly.csv', header = None)
weight.columns = ticket_symbol
weight.to_csv('weight_weekly.csv')